In [ ]:
import GPy
import numpy as np
import matplotlib.pyplot as plt
import utils
from functools import partial
from utils import KernelFunction, KernelEnvironment, log_likelihood_reward
from utils import plot_kernel_function, compare_kernels
from gflownet import GFlowNet 
import torch.nn.functional as F
from torch.distributions import Categorical
import torch
from utils import ForwardPolicy, BackwardPolicy
import random
from utils import train

from evaluation import create_random_

In [3]:
def create_env(batch_size=64):
    return KernelEnvironment(
    batch_size=batch_size,
    max_trajectory_length=MAX_LEN,
    log_reward=log_reward_fn
)

In [16]:
ll = -1#
while ll < 0:
    true_kernel = create_random_kernel()
    X, Y, true_kernel_str = utils.generate_gp_data(true_kernel, input_dim=1, n_points=30, noise_var=1e-4)
    X_test, Y_test, _ = utils.generate_gp_data(true_kernel, input_dim=1, n_points=30, noise_var=1e-4)
    ll = utils.evaluate_likelihood(true_kernel, X, Y, runtime=False)

print("True Kernel:", true_kernel_str, "Log Marginal Likelihood:", utils.evaluate_likelihood(true_kernel, X, Y, runtime=False))
print("True Kernel:", true_kernel_str, "Log Marginal Likelihood:", utils.evaluate_likelihood(true_kernel, X_test, Y_test, runtime=False))

#lt.scatter(X, Y, color='black', s=10, label='Data Points')

  -> Randomizing 'RBF' params...
  -> Randomizing 'Constant' params...
  -> Randomizing 'Constant' params...
True Kernel: ((RBF({'lengthscale': 1.225, 'variance': 1.368}) + Constant({'variance': 1.174})) * Constant({'variance': 0.652})) Log Marginal Likelihood: 48.40710578678093
True Kernel: ((RBF({'lengthscale': 1.225, 'variance': 1.368}) + Constant({'variance': 1.174})) * Constant({'variance': 0.652})) Log Marginal Likelihood: 48.110255059698595


In [ ]:

epochs = 100
BATCH_SIZE = 64
MAX_LEN = 4
lr = 1e-3

log_reward_fn = partial(utils.log_likelihood_reward, X, Y)
env = create_env()

forward_model = ForwardPolicy(input_dim=MAX_LEN, output_dim=env.action_space_size, epsilon=0.5)
backward_model = BackwardPolicy()
criterion = 'db'

gflownet = GFlowNet(
    forward_flow=forward_model, 
    backward_flow=backward_model, 
    criterion=criterion 
)


gflownet, losses = train(
    gflownet=gflownet,
    create_env=create_env,
    epochs=epochs,
    batch_size=BATCH_SIZE,
    lr=lr,
    min_eps=1e-2,
    clamp_g= 10,
    use_scheduler=True
)